In [1]:
# !pip install -q transformers
# !pip install -q accelerate
# !pip install -q datasets
# !pip install -q bitsandbytes
# !pip install -q sentence-transformers
# !pip install -q faiss-gpu
# !pip install -q langchain

#### you need to acquire a [GitHub personal access token](https://github.com/settings/tokens?type=beta) to access the GitHub API.

In [2]:
from getpass import getpass

ACCESS_TOKEN = getpass("YOUR GITHUB_PERSONAL_TOKEN")

YOUR GITHUB_PERSONAL_TOKEN··········


In [3]:
from langchain.document_loaders import GitHubIssuesLoader

loader = GitHubIssuesLoader(repo="huggingface/peft",
                            access_token=ACCESS_TOKEN,
                            include_prs=False,
                            state="all")

docs = loader.load()

In [4]:
type(docs)

list

In [5]:
type(docs[0])

langchain_core.documents.base.Document

In [6]:
docs[0].metadata

{'url': 'https://github.com/huggingface/peft/issues/1497',
 'title': 'Cannot load adapters from Peft.from_pretrained',
 'creator': 'dame-cell',
 'created_at': '2024-02-21T13:37:31Z',
 'comments': 1,
 'state': 'open',
 'labels': [],
 'assignee': None,
 'milestone': None,
 'locked': False,
 'number': 1497,
 'is_pull_request': False}

In [7]:
docs[0].page_content

'### System Info\n\nSo i was trying to run this code and it downloads the base model , but when i try to load the model with the adapter it gives me an error . \r\nIs it some thing wrong with what i did ?\r\n\r\nnote : I am using my own adapters not the one provided in the examples \r\n\r\n\n\n### Who can help?\n\n@pacman100 @younesbelkada @sayakpaul \n\n### Information\n\n- [X] The official example scripts\n- [ ] My own modified scripts\n\n### Tasks\n\n- [X] An officially supported task in the `examples` folder\n- [ ] My own task or dataset (give details below)\n\n### Reproduction\n\n```python\r\nimport os\r\n\r\nos.environ["CUDA_VISIBLE_DEVICES"] = "1"\r\nfrom peft import PeftConfig, PeftModel\r\nfrom peft import PeftModel, PeftConfig\r\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\r\nimport torch\r\nimport random\r\n\r\npeft_model_id = "my-adapter-path"\r\ndevice = "cuda"\r\nconfig = PeftConfig.from_pretrained(peft_model_id)\r\nmodel = AutoModelForCausalLM.from_pretr

In [8]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(docs)

In [9]:
chunked_docs[0]

Document(page_content='### System Info\n\nSo i was trying to run this code and it downloads the base model , but when i try to load the model with the adapter it gives me an error . \r\nIs it some thing wrong with what i did ?\r\n\r\nnote : I am using my own adapters not the one provided in the examples \r\n\r\n\n\n### Who can help?\n\n@pacman100 @younesbelkada @sayakpaul \n\n### Information\n\n- [X] The official example scripts\n- [ ] My own modified scripts\n\n### Tasks', metadata={'url': 'https://github.com/huggingface/peft/issues/1497', 'title': 'Cannot load adapters from Peft.from_pretrained', 'creator': 'dame-cell', 'created_at': '2024-02-21T13:37:31Z', 'comments': 1, 'state': 'open', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 1497, 'is_pull_request': False})

### Create the Embeddings + Retriever

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

# embeddings_model.client.tokenizer.pad_token = embeddings_model.client.tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(chunked_docs, embeddings_model)

In [13]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={'k': 4})

### Load qunatized model

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)

In [25]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### Setup the LLM chain

In [26]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(model=model,
                                    tokenizer=tokenizer,
                                    task="text-generation",
                                    temperature=0.2,
                                    do_sample=True,
                                    repetition_penalty=1.1,
                                    return_full_text=True,
                                    max_new_tokens=400)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [27]:
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

In [28]:
prompt = PromptTemplate(input_variables=["context", "question"],
                        template=prompt_template)

In [29]:
llm_chain = prompt | llm | StrOutputParser()

In [30]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = ({"context": retriever,
              "question": RunnablePassthrough()
              } | llm_chain)

### Compare the results

In [31]:
question = "How do you combine multiple adapters?"

In [32]:
llm_chain.invoke({"context": "",
                  "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


" To combine multiple adapters, you need to ensure that they are compatible with each other and the devices you want to connect. Here's how you can do it:\n\n1. Identify the type of adapter(s) you need: Determine which adapter(s) will allow you to connect the device(s) you want to use with the port(s) available on your computer or other device. For example, if you want to connect a USB-C device to an older laptop with a USB-A port, you'll need a USB-C to USB-A adapter.\n\n2. Check compatibility: Make sure the adapters you choose are compatible with each other and the devices you want to connect. For example, if you have a USB-C to HDMI adapter and want to connect it to a USB-C hub, make sure the hub has an HDMI output that is compatible with the adapter.\n\n3. Connect the adapters: Plug one adapter into another until all the necessary connections are made. For example, if you're connecting a USB-C device to an older laptop with a USB-A port, you might need a USB-C to USB-C adapter to c

In [33]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


' Currently, to obtain outputs from multiple adapters, you need to load and run each adapter separately. However, there are a few ways you can potentially combine multiple adapters:\n\n  1. Parallel processing: You can run each adapter in parallel using multi-threading or distributed computing frameworks like Dask or Ray. This will allow you to process multiple inputs simultaneously and reduce overall computation time.\n\n  2. Ensemble learning: You can train multiple adapters on different tasks or subsets of data and then combine their outputs through a weighted average, majority vote, or other ensemble methods. This can improve the accuracy and robustness of your predictions.\n\n  3. Modular architecture: Some researchers suggest creating a modular architecture where each adapter is responsible for a specific task or feature extraction. The final prediction is made by combining the outputs of all the adapters. This approach allows you to easily add or remove adapters as needed and al